In [1]:
import requests
import json
import os
import sqlalchemy
import pandas as pd
import mdapi_functions as md
from IPython.display import display, Markdown
import markdown
from bokeh.plotting import figure, show
from bokeh.models import (Span, TabPanel, Tabs, ColumnDataSource, DataCube,
                          GroupingInfo, StringFormatter, SumAggregator,
                          TableColumn, HoverTool)
from bokeh.io import output_notebook
from math import pi
from datetime import datetime
import datacite_api_functions as dcf
import numpy as np
import data_doc_helper as dh

In [ ]:
def cohort_total(ds):
    if ds in ["MHSDS", "IAPT", "CSDS"]:
      return "N/A - contains multiple auxiliary datasets"
    else:
       df = md.get_nhse_cohort_counts("CSDS")
    df["count"] = df["count"].apply(lambda x: int(x.replace("<10", "0")))
    return str(df["count"].sum())